In [1]:
import requests
import ollama
import numpy as np

In [2]:
"""
Проверяем доступность Ollama сервера через HTTP запрос
"""
try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        print("✅ Подключение к Ollama успешно!")
    else:
        print("❌ Ollama не отвечает")
except:
    print("❌ Не удалось подключиться к Ollama")

✅ Подключение к Ollama успешно!


In [3]:
"""
Используем ollama.list() для получения информации о загруженных моделях
"""
models = ollama.list()
models

ListResponse(models=[Model(model='qllama/bge-m3:latest', modified_at=datetime.datetime(2025, 12, 5, 15, 54, 52, 717068, tzinfo=TzInfo(10800)), digest='cf7e89be82c9775e6ac3f56c69b4063e9c0fb00cf4a9c839223dc5114e61b94a', size=634553992, details=ModelDetails(parent_model='', format='gguf', family='bert', families=['bert'], parameter_size='566.70M', quantization_level='Q8_0'))])

In [4]:
"""
Преобразуем текст в векторное представление с помощью BGE-M3 модели через ollama клиент
"""
def get_responce(in_prompt):
    response = ollama.embeddings(
        model='qllama/bge-m3:latest',  # или другая модель
        prompt=in_prompt
    )
    return np.array(response['embedding'])

In [5]:
"""
Создаем векторные представления для трех слов:
1. 'Кошка' - базовое понятие
2. 'Окошко' - несвязанное понятие  
3. 'Котенок' - семантически близкое к 'кошка'
"""
koshka = get_responce('Кошка')
okoshko = get_responce('Окошко')
kotenok = get_responce('Котенок')

In [6]:
"""
Используем sklearn для расчета семантической близости векторов
Ожидаемые результаты:
- Кошка-Окошко: низкое сходство (~0.4)
- Кошка-Котенок: высокое сходство (~0.65)
- Котенок-Окошко: среднее сходство (~0.50)
"""
from sklearn.metrics.pairwise import cosine_similarity

print("=" * 60)
print('Кошка и Окошко')
print(cosine_similarity(koshka.reshape(1, -1), okoshko.reshape(1, -1))[0][0])
print("=" * 60)
print('Кошка и Котенок')
print(cosine_similarity(koshka.reshape(1, -1), kotenok.reshape(1, -1))[0][0])
print("=" * 60)
print('Котенок и Окошко')
print(cosine_similarity(okoshko.reshape(1, -1), kotenok.reshape(1, -1))[0][0])

Кошка и Окошко
0.41724133690839865
Кошка и Котенок
0.6623491981748755
Котенок и Окошко
0.5111808452136047


In [7]:
"""
Прямой HTTP запрос к Ollama API без использования ollama клиента
"""
def generate(prompt, model="qllama/bge-m3:latest"):
    response = requests.post("http://localhost:11434/api/embeddings",
        json={
            "model": model,
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json()

In [8]:
"""
Получаем эмбеддинг для слова 'Кошка' через прямой HTTP запрос
"""
generate('Кошка')

{'embedding': [-0.3463195562362671,
  0.963005542755127,
  -0.8723657727241516,
  0.18182209134101868,
  -0.5610747337341309,
  -0.24971292912960052,
  -0.8265190720558167,
  0.8923777937889099,
  0.19591599702835083,
  -0.35494518280029297,
  0.9908009767532349,
  -0.24472101032733917,
  0.3922806978225708,
  -0.14730018377304077,
  -0.09734413027763367,
  -1.4533438682556152,
  0.08721461892127991,
  0.4264625310897827,
  -0.12287648022174835,
  -0.700105607509613,
  -0.5650827288627625,
  -1.0256046056747437,
  0.8529561161994934,
  -0.3246909976005554,
  -0.12112575769424438,
  0.2962391972541809,
  0.05808611214160919,
  -0.9106168746948242,
  -0.5312421321868896,
  -0.34768471121788025,
  -0.33164307475090027,
  1.1188639402389526,
  0.14461824297904968,
  -1.447326898574829,
  -1.1414653062820435,
  -1.994835376739502,
  -0.7025644183158875,
  -1.2974365949630737,
  -1.7932014465332031,
  0.5771868228912354,
  -0.03178169205784798,
  -0.3756571412086487,
  0.06385166198015213,
 